# Clustering Toronto Neighborhoods using geopy and folium

In [1]:
import numpy as np # library to handle data in a vectorized manner
!pip install geopy
!pip install folium
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Requirement not upgraded as not directly required: geopy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
    100% |████████████████████████████████| 92kB 7.8MB/s eta 0:00:01
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from

In [2]:
#create an empty data frame
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
toronto = pd.DataFrame(columns=column_names)
toronto #check the df

URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')

#pulll out the table elements and fill out the dataframe, clean up of newline stings on the way
for items in soup.find_all('tr')[1::1]:
    data = items.find_all('td')
    try:
        post=data[0].text
        postcode =post.replace("\n","Not assigned") 
        bor=data[1].text
        borough =bor.replace("\n","Not assigned") 
        neigh=data[2].text
        neighbourhood = neigh.replace("\n", "");
    except IndexError:pass
    toronto = toronto.append({'PostalCode': postcode,
                              'Borough': borough,
                              'Neighborhood': neighbourhood}, ignore_index=True)

#check the table
    toronto.head()

In [13]:
#clean up the table per requirement
#1 remove 'Not assigned Boroughs'
toronto=toronto[(toronto.Borough != "Not assigned")]

#2 group by postalcode and borough
toronto1=toronto.groupby(by=('PostalCode', 'Borough'))['Neighborhood'].apply(', '.join).reset_index()
toronto1=toronto1[(toronto1.Borough != "BNot assigned")]#clean up remaining non assigned boroughs
#3 asign neighborhoods which are not assigned
toronto1['Neighborhood'] = np.where(toronto1['Neighborhood'] == 'Not assigned', toronto1['Borough'], toronto1['Neighborhood'])
toronto1.head(10)

PostalCode      Borough                                     Neighborhood
0        M1B  Scarborough                                   Rouge, Malvern
1        M1C  Scarborough           Highland Creek, Rouge Hill, Port Union
2        M1E  Scarborough                Guildwood, Morningside, West Hill
3        M1G  Scarborough                                           Woburn
4        M1H  Scarborough                                        Cedarbrae
5        M1J  Scarborough                              Scarborough Village
6        M1K  Scarborough      East Birchmount Park, Ionview, Kennedy Park
7        M1L  Scarborough                  Clairlea, Golden Mile, Oakridge
8        M1M  Scarborough  Cliffcrest, Cliffside, Scarborough Village West
9        M1N  Scarborough                      Birch Cliff, Cliffside West

In [4]:

import types
import pandas as pd
from ibm_botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.

# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [18]:
toronto_new=pd.merge(toronto1, df_data_1, left_on = 'PostalCode', right_on = 'Postal Code', how = 'left')
toronto_new=toronto_new[(toronto_new.PostalCode != "NLNot assigned")]#clean up remaining non assigned postalcode
toronto_new=toronto_new[(toronto_new.Borough !="NLNot assigned")]#clean up remaining non assigned boroughs



In [19]:
toronto_new

PostalCode           Borough  \
0          M1B       Scarborough   
1          M1C       Scarborough   
2          M1E       Scarborough   
3          M1G       Scarborough   
4          M1H       Scarborough   
5          M1J       Scarborough   
6          M1K       Scarborough   
7          M1L       Scarborough   
8          M1M       Scarborough   
9          M1N       Scarborough   
10         M1P       Scarborough   
11         M1R       Scarborough   
12         M1S       Scarborough   
13         M1T       Scarborough   
14         M1V       Scarborough   
15         M1W       Scarborough   
16         M1X       Scarborough   
17         M2H        North York   
18         M2J        North York   
19         M2K        North York   
20         M2L        North York   
21         M2M        North York   
22         M2N        North York   
23         M2P        North York   
24         M2R        North York   
25         M3A        North York   
26         M3B        North York   
27         M3C        North York   
28         M3H        North York   
29         M3J        North York   
30         M3K        North York   
31         M3L        North York   
32         M3M        North York   
33         M3N        North York   
34         M4A        North York   
35         M4B         East York   
36         M4C         East York   
37         M4E      East Toronto   
38         M4G         East York   
39         M4H         East York   
40         M4J         East York   
41         M4K      East Toronto   
42         M4L      East Toronto   
43         M4M      East Toronto   
44         M4N   Central Toronto   
45         M4P   Central Toronto   
46         M4R   Central Toronto   
47         M4S   Central Toronto   
48         M4T   Central Toronto   
49         M4V   Central Toronto   
50         M4W  Downtown Toronto   
51         M4X  Downtown Toronto   
52         M4Y  Downtown Toronto   
53         M5A  Downtown Toronto   
54         M5B  Downtown Toronto   
55         M5C  Downtown Toronto   
56         M5E  Downtown Toronto   
57         M5G  Downtown Toronto   
58         M5H  Downtown Toronto   
59         M5J  Downtown Toronto   
60         M5K  Downtown Toronto   
61         M5L  Downtown Toronto   
62         M5M        North York   
63         M5N   Central Toronto   
64         M5P   Central Toronto   
65         M5R   Central Toronto   
66         M5S  Downtown Toronto   
67         M5T  Downtown Toronto   
68         M5V  Downtown Toronto   
69         M5W  Downtown Toronto   
70         M5X  Downtown Toronto   
71         M6A        North York   
72         M6B        North York   
73         M6C              York   
74         M6E              York   
75         M6G  Downtown Toronto   
76         M6H      West Toronto   
77         M6J      West Toronto   
78         M6K      West Toronto   
79         M6L        North York   
80         M6M              York   
81         M6N              York   
82         M6P      West Toronto   
83         M6R      West Toronto   
84         M6S      West Toronto   
85         M7A      Queen's Park   
86         M7R       Mississauga   
87         M7Y      East Toronto   
88         M8V         Etobicoke   
89         M8W         Etobicoke   
90         M8X         Etobicoke   
91         M8Y         Etobicoke   
92         M8Z         Etobicoke   
93         M9A         Etobicoke   
94         M9B         Etobicoke   
95         M9C         Etobicoke   
96         M9L        North York   
97         M9M        North York   
98         M9N              York   
99         M9P         Etobicoke   
100        M9R         Etobicoke   
101        M9V         Etobicoke   
102        M9W         Etobicoke   

                                          Neighborhood Postal Code   Latitude  \
0                                       Rouge, Malvern         M1B  43.806686   
1               Highland Creek, Rouge Hill, Port Union         M1C  43.784535   
2               

<b> <H2> Below is the section for the Q3 <b>

Create a map of Toronto with neighbourhoods 

In [20]:
address='Toronto, Canada'

geolocator=Nominatim(user_agent="toronto_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('the geo coordinate of Toronto are {}, {}.'.format(latitude, longitude))


the geo coordinate of Toronto are 43.653963, -79.387207.


In [21]:
#use map of Toronto using latitue and longitude
map_toronto=folium.Map(location=[latitude, longitude], zoom_start=10)

In [22]:


#adding markers
for lat, lng, borough, neighborhood in  zip(toronto_new['Latitude'], toronto_new['Longitude'], toronto_new['Borough'], toronto_new['Neighborhood']):
    label='{}, {}'.format(neighborhood, borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF6347',
        fill_opacity=0.7
        #, parse_html=False
        ).add_to(map_toronto)
                                           
map_toronto

<b> Define Foursquare Credentials and Version </b>

In [24]:
# @hidden_cell
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XU0PTBX2XJJ0HNJEBFUV242ORAAY3Z450TPDBCMDN0ALODCA
CLIENT_SECRET:WKEXUIY2VAQRCM1KFWYJBDUDOLIUS4XJO1QK01LJC4WLS43A


<H3> Explore Neighborhood </H3>

Getting neighbourhood names, venues and locations

In [31]:
neighborhood_latitude = toronto_new['Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_new['Longitude'] # neighborhood longitude value

neighborhood_name = toronto_new['Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of 0                                         Rouge, Malvern
1                 Highland Creek, Rouge Hill, Port Union
2                      Guildwood, Morningside, West Hill
3                                                 Woburn
4                                              Cedarbrae
5                                    Scarborough Village
6            East Birchmount Park, Ionview, Kennedy Park
7                        Clairlea, Golden Mile, Oakridge
8        Cliffcrest, Cliffside, Scarborough Village West
9                            Birch Cliff, Cliffside West
10     Dorset Park, Scarborough Town Centre, Wexford ...
11                                     Maryvale, Wexford
12                                             Agincourt
13               Clarks Corners, Sullivan, Tam O'Shanter
14     Agincourt North, L'Amoreaux East, Milliken, St...
15                         L'Amoreaux West, Steeles West
16                                           Upper Roug

In [35]:
# create the GET request URL

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
 

'https://api.foursquare.com/v2/venues/explore?&client_id=XU0PTBX2XJJ0HNJEBFUV242ORAAY3Z450TPDBCMDN0ALODCA&client_secret=WKEXUIY2VAQRCM1KFWYJBDUDOLIUS4XJO1QK01LJC4WLS43A&v=20180605&ll=0      43.806686\n1      43.784535\n2      43.763573\n3      43.770992\n4      43.773136\n5      43.744734\n6      43.727929\n7      43.711112\n8      43.716316\n9      43.692657\n10     43.757410\n11     43.750072\n12     43.794200\n13     43.781638\n14     43.815252\n15     43.799525\n16     43.836125\n17     43.803762\n18     43.778517\n19     43.786947\n20     43.757490\n21     43.789053\n22     43.770120\n23     43.752758\n24     43.782736\n25     43.753259\n26     43.745906\n27     43.725900\n28     43.754328\n29     43.767980\n30     43.737473\n31     43.739015\n32     43.728496\n33     43.761631\n34     43.725882\n35     43.706397\n36     43.695344\n37     43.676357\n38     43.709060\n39     43.705369\n40     43.685347\n41     43.679557\n42     43.668999\n43     43.659526\n44     43.728020\n45     

In [36]:
results = requests.get(url).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
#creating list of toronto venues
toronto_venues = getNearbyVenues(names=toronto_new['Neighborhood'],
                                   latitudes=toronto_new['Latitude'],
                                   longitudes=toronto_new['Longitude']
                                  )



Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

In [39]:
#check the size of the df
print(toronto_venues.shape)
toronto_venues.head()

(2252, 7)


Neighborhood  Neighborhood Latitude  \
0                          Rouge, Malvern              43.806686   
1  Highland Creek, Rouge Hill, Port Union              43.784535   
2  Highland Creek, Rouge Hill, Port Union              43.784535   
3       Guildwood, Morningside, West Hill              43.763573   
4       Guildwood, Morningside, West Hill              43.763573   

   Neighborhood Longitude                            Venue  Venue Latitude  \
0              -79.194353                          Wendy's       43.807448   
1              -79.160497            Royal Canadian Legion       43.782533   
2              -79.160497        Affordable Toronto Movers       43.787919   
3              -79.188711  Swiss Chalet Rotisserie & Grill       43.767697   
4              -79.188711                G & G Electronics       43.765309   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.163085                   Bar  
2       -79.162977         Moving Target  
3       -79.189914           Pizza Place  
4       -79.191537     Electronics Store

In [46]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


In [41]:
#number of venues
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Adelaide, King, Richmond                                              100   
Agincourt                                                               4   
Agincourt North, L'Amoreaux East, Milliken, Ste...                      2   
Albion Gardens, Beaumond Heights, Humbergate, J...                     10   
Alderwood, Long Branch                                                 10   
Bathurst Manor, Downsview North, Wilson Heights                        17   
Bayview Village                                                         4   
Bedford Park, Lawrence Manor East                                      25   
Berczy Park                                                            54   
Birch Cliff, Cliffside West                                             4   
Bloordale Gardens, Eringate, Markland Wood, Old...                      6   
Brockton, Exhibition Place, Parkdale Village                           19   
Business Reply Mail Processing Centre 969 Eastern                      17   
CFB Toronto, Downsview East                                             4   
CN Tower, Bathurst Quay, Island airport, Harbou...                     14   
Cabbagetown, St. James Town                                            47   
Caledonia-Fairbanks                                                     6   
Canada Post Gateway Processing Centre                                  11   
Cedarbrae                                                               8   
Central Bay Street                                                     82   
Chinatown, Grange Park, Kensington Market                             100   
Christie                                                               15   
Church and Wellesley                                                   87   
Clairlea, Golden Mile, Oakridge                                        10   
Clarks Corners, Sullivan, Tam O'Shanter                                 9   
Cliffcrest, Cliffside, Scarborough Village West                         3   
Cloverdale, Islington, Martin Grove, Princess G...                      2   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             36   
Davisville North                                                       10   
Deer Park, Forest Hill SE, Rathnelly, South Hil...                     14   
Del Ray, Keelesdale, Mount Dennis, Silverthorn                          5   
Design Exchange, Toronto Dominion Centre                              100   
Don Mills North                                                         5   
Dorset Park, Scarborough Town Centre, Wexford H...                      7   
Dovercourt Village, Dufferin                                           19   
Downsview Central                                                       3   
Downsview Northwest                                                     5   
Downsview West                                                          4   
East Birchmount Park, Ionview, Kennedy Park                             6   
East Toronto                                                            2   
Emery, Humberlea                                                        1   
Fairview, Henry Farm, Oriole                                           63   
First Canadian Place, Underground city                                100   
Flemingdon Park, Don Mills South                                       20   
Forest Hill North, Forest Hill West                                     4   
Glencairn                                                               4   
Guildwood, Morningside, West Hill                                       6   
Harbord, University of Toronto                                         34   
Harbourfront East, Toronto Islands, Union Station                     100   
Harbourfront, Regent Park                       

In [ ]:
#Analyse Neighborhood
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

In [59]:
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.head()

Yoga Studio  Accessories Store  Adult Boutique  Afghan Restaurant  Airport  \
0            0                  0               0                  0        0   
1            0                  0               0                  0        0   
2            0                  0               0                  0        0   
3            0                  0               0                  0        0   
4            0                  0               0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  \
0                 0                    0             0         0   
1                 0                    0             0         0   
2                 0                    0             0         0   
3                 0                    0             0         0   
4                 0                    0             0         0   

   Arepa Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0                 0            0           0                    0   
1                 0            0           0                    0   
2                 0            0           0                    0   
3                 0            0           0                    0   
4                 0            0           0                    0   

   Asian Restaurant  Athletics & Sports  Auto Garage  Auto Workshop  \
0                 0                   0            0              0   
1                 0                   0            0              0   
2                 0                   0            0              0   
3                 0                   0            0              0   
4                 0                   0            0              0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  Baseball Field  \
0          0           0           0       0     0    0               0   
1          0           0           0       0     0    1               0   
2          0           0           0       0     0    0               0   
3          0           0           0       0     0    0               0   
4          0           0           0       0     0    0               0   

   Baseball Stadium  Basketball Court  Basketball Stadium  Beach  Beer Bar  \
0                 0                 0                   0      0         0   
1                 0                 0                   0      0         0   
2                 0                 0                   0      0         0   
3                 0                 0                   0      0         0   
4                 0                 0                   0      0         0   

   Beer Store  Belgian Restaurant  Bike Shop  Bistro  Board Shop  \
0           0                   0          0       0           0   
1           0                   0          0       0           0   
2           0                   0          0       0           0   
3           0                   0          0       0           0   
4           0                   0          0       0           0   

   Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  \
0              0          0         0                     0               0   
1              0          0         0                     0               0   
2              0          0         0                     0               0   
3              0          0         0                     0               0   
4              0          0         0                     0               0   

   Brewery  Bridal Shop  Bub

In [64]:
# move neighborhood column to the first column
cols = toronto_onehot.columns.tolist()
n = int(cols.index('Neighborhood'))
cols = [cols[n]] + cols[:n] + cols[n+1:]
toronto_onehot = toronto_onehot[cols]

toronto_onehot.head()

Neighborhood  Yoga Studio  Accessories Store  \
0                          Rouge, Malvern            0                  0   
1  Highland Creek, Rouge Hill, Port Union            0                  0   
2  Highland Creek, Rouge Hill, Port Union            0                  0   
3       Guildwood, Morningside, West Hill            0                  0   
4       Guildwood, Morningside, West Hill            0                  0   

   Adult Boutique  Afghan Restaurant  Airport  Airport Food Court  \
0               0                  0        0                   0   
1               0                  0        0                   0   
2               0                  0        0                   0   
3               0                  0        0                   0   
4               0                  0        0                   0   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  \
0             0               0                0                 0   
1             0               0                0                 0   
2             0               0                0                 0   
3             0               0                0                 0   
4             0               0                0                 0   

   American Restaurant  Antique Shop  Aquarium  Arepa Restaurant  Art Gallery  \
0                    0             0         0                 0            0   
1                    0             0         0                 0            0   
2                    0             0         0                 0            0   
3                    0             0         0                 0            0   
4                    0             0         0                 0            0   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0           0                    0                 0                   0   
1           0                    0                 0                   0   
2           0                    0                 0                   0   
3           0                    0                 0                   0   
4           0                    0                 0                   0   

   Auto Garage  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  \
0            0              0          0           0           0       0   
1            0              0          0           0           0       0   
2            0              0          0           0           0       0   
3            0              0          0           0           0       0   
4            0              0          0           0           0       0   

   Bank  Bar  Baseball Field  Baseball Stadium  Basketball Court  \
0     0    0               0                 0                 0   
1     0    1               0                 0                 0   
2     0    0               0                 0                 0   
3     0    0               0                 0                 0   
4     0    0               0                 0                 0   

   Basketball Stadium  Beach  Beer Bar  Beer Store  Belgian Restaurant  \
0                   0      0         0           0                   0   
1                   0      0         0           0                   0   
2                   0      0         0           0                   0   
3                   0      0         0           0                   0   
4                   0      0         0           0                   0   

   Bike Shop  Bistro  Board Shop  Boat or Ferry  Bookstore  Boutique  \
0          0       0           0              0          0         0   
1          0       0           0              0          0         0   
2          0       0           0              0          0         0   
3          0       0           0              0          0         0   
4          0       0           0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  B

In [65]:
toronto_onehot.shape

(2252, 276)

In [66]:
#group beighborhoods
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

Neighborhood  Yoga Studio  \
0                           Adelaide, King, Richmond          0.0   
1                                          Agincourt          0.0   
2  Agincourt North, L'Amoreaux East, Milliken, St...          0.0   
3  Albion Gardens, Beaumond Heights, Humbergate, ...          0.0   
4                             Alderwood, Long Branch          0.0   

   Accessories Store  Adult Boutique  Afghan Restaurant  Airport  \
0                0.0             0.0                0.0      0.0   
1                0.0             0.0                0.0      0.0   
2                0.0             0.0                0.0      0.0   
3                0.0             0.0                0.0      0.0   
4                0.0             0.0                0.0      0.0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                 0.0           0.0             0.0              0.0   
1                 0.0           0.0             0.0              0.0   
2                 0.0           0.0             0.0              0.0   
3                 0.0           0.0             0.0              0.0   
4                 0.0           0.0             0.0              0.0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  \
0               0.0                 0.04           0.0       0.0   
1               0.0                 0.00           0.0       0.0   
2               0.0                 0.00           0.0       0.0   
3               0.0                 0.00           0.0       0.0   
4               0.0                 0.00           0.0       0.0   

   Arepa Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0               0.0         0.01        0.01                  0.0   
1               0.0         0.00        0.00                  0.0   
2               0.0         0.00        0.00                  0.0   
3               0.0         0.00        0.00                  0.0   
4               0.0         0.00        0.00                  0.0   

   Asian Restaurant  Athletics & Sports  Auto Garage  Auto Workshop  \
0              0.03                 0.0          0.0            0.0   
1              0.00                 0.0          0.0            0.0   
2              0.00                 0.0          0.0            0.0   
3              0.00                 0.0          0.0            0.0   
4              0.00                 0.0          0.0            0.0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank   Bar  Baseball Field  \
0        0.0         0.0         0.0    0.03   0.0  0.03             0.0   
1        0.0         0.0         0.0    0.00   0.0  0.00             0.0   
2        0.0         0.0         0.0    0.00   0.0  0.00             0.0   
3        0.0         0.0         0.0    0.00   0.0  0.00             0.0   
4        0.0         0.0         0.0    0.00   0.0  0.00             0.0   

   Baseball Stadium  Basketball Court  Basketball Stadium  Beach  Beer Bar  \
0               0.0               0.0                 0.0    0.0       0.0   
1               0.0               0.0                 0.0    0.0       0.0   
2               0.0               0.0                 0.0    0.0       0.0   
3               0.0               0.0                 0.0    0.0       0.0   
4               0.0               0.0                 0.0    0.0       0.0   

   Beer Store  Belgian Restaurant  Bike Shop  Bistro  Board Shop  \
0         0.0                 0.0        0.0     0.0         0.0   
1         0.0                 0.0        0.0     0.0         0.0   
2         0.0                 0.0        0.0     0.0         0.0   
3         0.1                 0.0        0.0     0.0         0.0   
4         0.0                 0.0        0.0     0.0         0.0   

   Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  \
0            0.0       0.02       0.0                  0.01            0.02   
1            0.0       0.00       0.0  

In [67]:
toronto_grouped.shape

(99, 276)

In [69]:
#get top 10 venues per neighbourhoods
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04
5     Asian Restaurant  0.03
6                  Bar  0.03
7                  Gym  0.03
8               Bakery  0.03
9           Restaurant  0.03


----Agincourt----
                             venue  freq
0                     Skating Rink  0.25
1                   Sandwich Place  0.25
2                           Lounge  0.25
3                   Breakfast Spot  0.25
4               Miscellaneous Shop  0.00
5              Monument / Landmark  0.00
6  Molecular Gastronomy Restaurant  0.00
7       Modern European Restaurant  0.00
8                Mobile Phone Shop  0.00
9                      Yoga Studio  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                             venue  freq
0                       Playground   0.5
1                           

                venue  freq
0                Café  0.20
1       Grocery Store  0.20
2                Park  0.13
3           Nightclub  0.07
4               Diner  0.07
5          Restaurant  0.07
6  Italian Restaurant  0.07
7          Baby Store  0.07
8         Coffee Shop  0.07
9   Convenience Store  0.07


----Church and Wellesley----
                  venue  freq
0   Japanese Restaurant  0.07
1           Coffee Shop  0.06
2      Sushi Restaurant  0.06
3               Gay Bar  0.05
4          Burger Joint  0.03
5            Restaurant  0.03
6             Gastropub  0.02
7  Fast Food Restaurant  0.02
8           Men's Store  0.02
9                  Café  0.02


----Clairlea, Golden Mile, Oakridge----
                  venue  freq
0              Bus Line   0.2
1                Bakery   0.2
2          Soccer Field   0.1
3  Fast Food Restaurant   0.1
4         Metro Station   0.1
5                  Park   0.1
6           Bus Station   0.1
7          Intersection   0.1
8           Yoga St

                 venue  freq
0          Coffee Shop  0.10
1     Asian Restaurant  0.10
2           Beer Store  0.10
3                  Gym  0.10
4        Grocery Store  0.05
5           Restaurant  0.05
6   Chinese Restaurant  0.05
7       Sandwich Place  0.05
8       Clothing Store  0.05
9  Sporting Goods Shop  0.05


----Forest Hill North, Forest Hill West----
                             venue  freq
0                    Jewelry Store  0.25
1                 Sushi Restaurant  0.25
2                         Bus Line  0.25
3                            Trail  0.25
4                      Yoga Studio  0.00
5                    Movie Theater  0.00
6              Monument / Landmark  0.00
7  Molecular Gastronomy Restaurant  0.00
8       Modern European Restaurant  0.00
9                Mobile Phone Shop  0.00


----Glencairn----
                             venue  freq
0              Japanese Restaurant  0.25
1                 Asian Restaurant  0.25
2                             Park  0.25


                             venue  freq
0                 Basketball Court  0.25
1       Construction & Landscaping  0.25
2                           Bakery  0.25
3                             Park  0.25
4               Miscellaneous Shop  0.00
5                            Motel  0.00
6              Monument / Landmark  0.00
7  Molecular Gastronomy Restaurant  0.00
8       Modern European Restaurant  0.00
9                Mobile Phone Shop  0.00


----Maryvale, Wexford----
                             venue  freq
0                   Sandwich Place  0.17
1                   Breakfast Spot  0.17
2                           Bakery  0.17
3                       Smoke Shop  0.17
4                      Auto Garage  0.17
5                    Shopping Mall  0.17
6                      Yoga Studio  0.00
7                Mobile Phone Shop  0.00
8              Monument / Landmark  0.00
9  Molecular Gastronomy Restaurant  0.00


----Moore Park, Summerhill East----
                             ven

                     venue  freq
0         Greek Restaurant  0.24
1              Coffee Shop  0.10
2           Ice Cream Shop  0.07
3                Bookstore  0.05
4       Italian Restaurant  0.05
5          Bubble Tea Shop  0.02
6             Liquor Store  0.02
7   Furniture / Home Store  0.02
8  Fruit & Vegetable Store  0.02
9                      Spa  0.02


----The Junction North, Runnymede----
                             venue  freq
0                      Pizza Place  0.25
1                         Bus Line  0.25
2                Convenience Store  0.25
3                    Grocery Store  0.25
4                Mobile Phone Shop  0.00
5                    Movie Theater  0.00
6                            Motel  0.00
7              Monument / Landmark  0.00
8  Molecular Gastronomy Restaurant  0.00
9       Modern European Restaurant  0.00


----The Kingsway, Montgomery Road, Old Mill North----
                             venue  freq
0                            River  0.33
1       

In [70]:
#sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:
#creating a new dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                           Adelaide, King, Richmond           Coffee Shop   
1                                          Agincourt        Breakfast Spot   
2  Agincourt North, L'Amoreaux East, Milliken, St...            Playground   
3  Albion Gardens, Beaumond Heights, Humbergate, ...         Grocery Store   
4                             Alderwood, Long Branch           Pizza Place   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                  Café   American Restaurant       Thai Restaurant   
1                Lounge          Skating Rink        Sandwich Place   
2                  Park         Women's Store             Drugstore   
3          Liquor Store           Coffee Shop        Sandwich Place   
4              Pharmacy          Skating Rink          Dance Studio   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0            Steakhouse                Bakery      Asian Restaurant   
1   Dumpling Restaurant        Discount Store               Dog Run   
2    Dim Sum Restaurant                 Diner        Discount Store   
3            Beer Store  Fast Food Restaurant   Fried Chicken Joint   
4           Coffee Shop                  Pool                   Pub   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                   Gym        Clothing Store                    Bar  
1      Doner Restaurant            Donut Shop              Drugstore  
2               Dog Run      Doner Restaurant             Donut Shop  
3           Pizza Place              Pharmacy      Electronics Store  
4        Sandwich Place                   Gym        Airport Service

<H2> Clustering Neighborhoods

In [72]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 4, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [77]:
toronto_grouped_clustering.head()

Yoga Studio  Accessories Store  Adult Boutique  Afghan Restaurant  Airport  \
0          0.0                0.0             0.0                0.0      0.0   
1          0.0                0.0             0.0                0.0      0.0   
2          0.0                0.0             0.0                0.0      0.0   
3          0.0                0.0             0.0                0.0      0.0   
4          0.0                0.0             0.0                0.0      0.0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                 0.0           0.0             0.0              0.0   
1                 0.0           0.0             0.0              0.0   
2                 0.0           0.0             0.0              0.0   
3                 0.0           0.0             0.0              0.0   
4                 0.0           0.0             0.0              0.0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  \
0               0.0                 0.04           0.0       0.0   
1               0.0                 0.00           0.0       0.0   
2               0.0                 0.00           0.0       0.0   
3               0.0                 0.00           0.0       0.0   
4               0.0                 0.00           0.0       0.0   

   Arepa Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0               0.0         0.01        0.01                  0.0   
1               0.0         0.00        0.00                  0.0   
2               0.0         0.00        0.00                  0.0   
3               0.0         0.00        0.00                  0.0   
4               0.0         0.00        0.00                  0.0   

   Asian Restaurant  Athletics & Sports  Auto Garage  Auto Workshop  \
0              0.03                 0.0          0.0            0.0   
1              0.00                 0.0          0.0            0.0   
2              0.00                 0.0          0.0            0.0   
3              0.00                 0.0          0.0            0.0   
4              0.00                 0.0          0.0            0.0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank   Bar  Baseball Field  \
0        0.0         0.0         0.0    0.03   0.0  0.03             0.0   
1        0.0         0.0         0.0    0.00   0.0  0.00             0.0   
2        0.0         0.0         0.0    0.00   0.0  0.00             0.0   
3        0.0         0.0         0.0    0.00   0.0  0.00             0.0   
4        0.0         0.0         0.0    0.00   0.0  0.00             0.0   

   Baseball Stadium  Basketball Court  Basketball Stadium  Beach  Beer Bar  \
0               0.0               0.0                 0.0    0.0       0.0   
1               0.0               0.0                 0.0    0.0       0.0   
2               0.0               0.0                 0.0    0.0       0.0   
3               0.0               0.0                 0.0    0.0       0.0   
4               0.0               0.0                 0.0    0.0       0.0   

   Beer Store  Belgian Restaurant  Bike Shop  Bistro  Board Shop  \
0         0.0                 0.0        0.0     0.0         0.0   
1         0.0                 0.0        0.0     0.0         0.0   
2         0.0                 0.0        0.0     0.0         0.0   
3         0.1                 0.0        0.0     0.0         0.0   
4         0.0                 0.0        0.0     0.0         0.0   

   Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  \
0            0.0       0.02       0.0                  0.01            0.02   
1            0.0       0.00       0.0                  0.00            0.25   
2            0.0       0.00       0.0                  0.00            0.00   
3            0.0       0.00       0.0                  0.00            0.00   
4            0.0       0.00       0.0                  0.00            0.00   

   Brewery  Bridal Sho

In [78]:
toronto_grouped_clustering.shape

(99, 275)

In [83]:
toronto_grouped['clusters']=kmeans.labels_

In [86]:
#join back neighborhoods and then boroughs and longitude and latitude
toronto_grouped= toronto_grouped[['Neighborhood', 'clusters']]
toronto_clustered=pd.merge(toronto_grouped, toronto_new , left_on = 'Neighborhood', right_on = 'Neighborhood', how = 'left').drop(['PostalCode', 'Postal Code'], axis = 1)

In [87]:
toronto_clustered.head()

Neighborhood  clusters  \
0                           Adelaide, King, Richmond         3   
1                                          Agincourt         3   
2  Agincourt North, L'Amoreaux East, Milliken, St...         4   
3  Albion Gardens, Beaumond Heights, Humbergate, ...         3   
4                             Alderwood, Long Branch         3   

            Borough   Latitude  Longitude  
0  Downtown Toronto  43.650571 -79.384568  
1       Scarborough  43.794200 -79.262029  
2       Scarborough  43.815252 -79.284577  
3         Etobicoke  43.739416 -79.588437  
4         Etobicoke  43.602414 -79.543484

In [88]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_clustered = toronto_clustered.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_clustered.head() # check the last columns!

Neighborhood  clusters  \
0                           Adelaide, King, Richmond         3   
1                                          Agincourt         3   
2  Agincourt North, L'Amoreaux East, Milliken, St...         4   
3  Albion Gardens, Beaumond Heights, Humbergate, ...         3   
4                             Alderwood, Long Branch         3   

            Borough   Latitude  Longitude 1st Most Common Venue  \
0  Downtown Toronto  43.650571 -79.384568           Coffee Shop   
1       Scarborough  43.794200 -79.262029        Breakfast Spot   
2       Scarborough  43.815252 -79.284577            Playground   
3         Etobicoke  43.739416 -79.588437         Grocery Store   
4         Etobicoke  43.602414 -79.543484           Pizza Place   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                  Café   American Restaurant       Thai Restaurant   
1                Lounge          Skating Rink        Sandwich Place   
2                  Park         Women's Store             Drugstore   
3          Liquor Store           Coffee Shop        Sandwich Place   
4              Pharmacy          Skating Rink          Dance Studio   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0            Steakhouse                Bakery      Asian Restaurant   
1   Dumpling Restaurant        Discount Store               Dog Run   
2    Dim Sum Restaurant                 Diner        Discount Store   
3            Beer Store  Fast Food Restaurant   Fried Chicken Joint   
4           Coffee Shop                  Pool                   Pub   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                   Gym        Clothing Store                    Bar  
1      Doner Restaurant            Donut Shop              Drugstore  
2               Dog Run      Doner Restaurant             Donut Shop  
3           Pizza Place              Pharmacy      Electronics Store  
4        Sandwich Place                   Gym        Airport Service

In [90]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_clustered['Latitude'], toronto_clustered['Longitude'], toronto_clustered['Neighborhood'], toronto_clustered['clusters']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<H2> Examine Some Clusters <H2>

In [92]:
#Cluster 1
toronto_clustered.loc[toronto_clustered['clusters'] == 0, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

clusters 1st Most Common Venue       2nd Most Common Venue  \
13         0                  Park                  Playground   
16         0                  Park               Women's Store   
46         0      Asian Restaurant                        Park   
57         0                 Field                        Park   
58         0              Bus Line                 Pizza Place   
62         0              Bus Line          Dim Sum Restaurant   
65         0                  Park  Construction & Landscaping   
67         0                   Gym                        Park   
88         0                 River                        Park   
98         0                  Bank                        Park   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
13               Airport              Bus Stop         Women's Store   
16              Pharmacy                Market  Fast Food Restaurant   
46   Japanese Restaurant                   Pub             Drugstore   
57          Hockey Arena                 Trail           Event Space   
58                  Park     Mobile Phone Shop             Drugstore   
62           Swim School                  Park            Donut Shop   
65      Basketball Court                Bakery         Women's Store   
67            Playground          Tennis Court         Women's Store   
88                  Pool             Drugstore    Dim Sum Restaurant   
98     Electronics Store         Women's Store        Discount Store   

          6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
13                   Donut Shop    Dim Sum Restaurant                 Diner   
16             Greek Restaurant          Gourmet Shop  Ethiopian Restaurant   
46                        Diner        Discount Store               Dog Run   
57         Ethiopian Restaurant   Empanada Restaurant    Falafel Restaurant   
58                        Diner        Discount Store               Dog Run   
62                        Diner        Discount Store               Dog Run   
65  Eastern European Restaurant               Dog Run      Doner Restaurant   
67                   Donut Shop    Dim Sum Restaurant                 Diner   
88                        Diner        Discount Store               Dog Run   
98                      Dog Run      Doner Restaurant            Donut Shop   

   9th Most Common Venue 10th Most Common Venue  
13        Discount Store                Dog Run  
16   Empanada Restaurant      Electronics Store  
46      Doner Restaurant             Donut Shop  
57     Electronics Store           Dessert Shop  
58      Doner Restaurant             Donut Shop  
62      Doner Restaurant          Women's Store  
65            Donut Shop              Drugstore  
67        Discount Store                Dog Run  
88      Doner Restaurant             Donut Shop  
98             Drugstore    Dumpling Restaurant

In [93]:
#Cluster 2
toronto_clustered.loc[toronto_clustered['clusters'] == 1, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

clusters 1st Most Common Venue 2nd Most Common Venue  \
72         1     Food & Drink Shop                  Park   
76         1  Fast Food Restaurant   Dumpling Restaurant   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
72  Fast Food Restaurant           Event Space  Ethiopian Restaurant   
76                 Diner        Discount Store               Dog Run   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
72    Falafel Restaurant   Empanada Restaurant     Electronics Store   
76      Doner Restaurant            Donut Shop             Drugstore   

          9th Most Common Venue 10th Most Common Venue  
72  Eastern European Restaurant     Dim Sum Restaurant  
76  Eastern European Restaurant            College Gym

In [94]:
#Cluster 3
toronto_clustered.loc[toronto_clustered['clusters'] == 2, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

clusters 1st Most Common Venue 2nd Most Common Venue  \
41         2        Baseball Field         Women's Store   
55         2        Baseball Field         Women's Store   

          3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
41  Eastern European Restaurant        Discount Store               Dog Run   
55  Eastern European Restaurant        Discount Store               Dog Run   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
41      Doner Restaurant            Donut Shop             Drugstore   
55      Doner Restaurant            Donut Shop             Drugstore   

   9th Most Common Venue 10th Most Common Venue  
41   Dumpling Restaurant      Electronics Store  
55   Dumpling Restaurant      Electronics Store

In [95]:
#Cluster 4
toronto_clustered.loc[toronto_clustered['clusters'] == 3, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

clusters   1st Most Common Venue     2nd Most Common Venue  \
0          3             Coffee Shop                      Café   
1          3          Breakfast Spot                    Lounge   
3          3           Grocery Store              Liquor Store   
4          3             Pizza Place                  Pharmacy   
5          3             Coffee Shop                  Pharmacy   
6          3     Japanese Restaurant        Chinese Restaurant   
7          3        Sushi Restaurant               Coffee Shop   
8          3             Coffee Shop              Cocktail Bar   
9          3         College Stadium                      Café   
10         3                Pharmacy                Beer Store   
11         3                    Café               Coffee Shop   
12         3             Yoga Studio             Auto Workshop   
14         3          Airport Lounge           Airport Service   
15         3              Restaurant               Coffee Shop   
17         3             Coffee Shop                     Hotel   
18         3        Hakka Restaurant                      Bank   
19         3             Coffee Shop                      Café   
20         3                     Bar                      Café   
21         3           Grocery Store                      Café   
22         3     Japanese Restaurant               Coffee Shop   
23         3                  Bakery                  Bus Line   
24         3             Pizza Place                  Pharmacy   
25         3            Intersection                     Motel   
26         3                    Bank               Golf Course   
27         3             Coffee Shop                      Café   
28         3             Pizza Place            Sandwich Place   
29         3                    Park                       Gym   
30         3             Coffee Shop                       Pub   
31         3              Restaurant     Check Cashing Service   
32         3             Coffee Shop                      Café   
33         3    Gym / Fitness Center                      Café   
34         3       Indian Restaurant        Chinese Restaurant   
35         3                Pharmacy               Supermarket   
36         3       Korean Restaurant                Food Truck   
37         3    Gym / Fitness Center            Discount Store   
38         3                    Bank             Moving Target   
39         3          Discount Store          Department Store   
42         3          Clothing Store      Fast Food Restaurant   
43         3             Coffee Shop                      Café   
44         3                     Gym          Asian Restaurant   
45         3                   Trail             Jewelry Store   
47         3     Rental Car Location               Pizza Place   
48         3                    Café       Japanese Restaurant   
49         3             Coffee Shop                  Aquarium   
50         3             Coffee Shop                    Bakery   
51         3                    Café        Mexican Restaurant   
52         3           Moving Target                       Bar   
53         3                 Dog Run  Mediterranean Restaurant   
54         3                    Café                  Pharmacy   
56         3             Pizza Place       Empanada Restaurant   
59         3             Social Club            Sandwich Place   
60         3      Chinese Restaurant      Fast Food Restaurant   
61         3          Clothing Store    Furniture / Home Store   
63         3             Coffee Shop       Sporting Goods Shop   
64         3                     Bar               Men's Store   
66         3             Auto Garage            Sandwich Place   
68         3     Sporting Goods Shop               Coffee Shop   
69         3               Drugstore       Rental Car Location   
70         3  Furniture / Home Store                       Bar   
71         3          Breakfast Spot        

In [96]:
#Cluster 5
toronto_clustered.loc[toronto_clustered['clusters'] == 4, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

clusters 1st Most Common Venue       2nd Most Common Venue  \
2          4            Playground                        Park   
79         4            Playground  Construction & Landscaping   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
2          Women's Store             Drugstore    Dim Sum Restaurant   
79         Women's Store   Dumpling Restaurant                 Diner   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
2                  Diner        Discount Store               Dog Run   
79        Discount Store               Dog Run      Doner Restaurant   

   9th Most Common Venue 10th Most Common Venue  
2       Doner Restaurant             Donut Shop  
79            Donut Shop              Drugstore

In [97]:
#Cluster 6
toronto_clustered.loc[toronto_clustered['clusters'] == 5, toronto_clustered.columns[[1] + list(range(5, toronto_clustered.shape[1]))]]

clusters 1st Most Common Venue 2nd Most Common Venue  \
40         5                  Park     Convenience Store   
74         5                  Park                 Trail   
92         5                  Park     Convenience Store   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
40         Women's Store   Dumpling Restaurant                 Diner   
74            Playground             Drugstore    Dim Sum Restaurant   
92         Women's Store   Dumpling Restaurant                 Diner   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
40        Discount Store               Dog Run      Doner Restaurant   
74                 Diner        Discount Store               Dog Run   
92        Discount Store               Dog Run      Doner Restaurant   

   9th Most Common Venue 10th Most Common Venue  
40            Donut Shop              Drugstore  
74      Doner Restaurant             Donut Shop  
92            Donut Shop              Drugstore

<b>Cluster 1 </b>- is mainly about Parks and Playgrounds

<b>Cluster 2 </b> - Food and Drink shops and Restaurants

<b>Cluster 3 </b> - Baseball Field is a strong feature along with Women's shoes

<b>Cluster 4 </b> - mixture of coffee shops and parks

<b>Cluster 5 </b> - focus on playgronds and parks (similiar to Cluster 1)

<b>Cluster 6 </b>- combination of Parks and Convenience Stores